# 😎 4. Now I Know Kung Fu

In [ ]:
import pykx as kx

## Traffic

In [ ]:
%%q
t:("IPSJS";enlist csv) 0: `$":data/traffic.csv";
t:`fecha xasc select avg carga by fecha from t where error=`N;
t

## Weather

In [ ]:
%%q
w:("DUIFFFFFFFF";enlist csv) 0: `$":data/weather.csv";
w:`fecha xasc select avg precipitacion by fecha+hora from w where not null precipitacion;
w

## All Together

In [ ]:
%%q
tw:aj[`fecha;t;w]
tw

## Model

In [ ]:
def model(traffic_weather):

    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    import matplotlib.pyplot as plt
    
    traffic_weather = traffic_weather.pd().reset_index()
    
    to_quarter = lambda x: int((x.hour * 60 + x.minute) / 15)
    traffic_weather['hora'] = traffic_weather['fecha'].dt.time.apply(to_quarter)
    
    X = traffic_weather[['hora', 'precipitacion']]
    y = traffic_weather['carga']
    
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
    
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    predictions = model.predict(X_test)
    
    plt.scatter(y_test, predictions, color='blue')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)  # Diagonal line for reference
    plt.xlabel('Actual Traffic Load')
    plt.ylabel('Predicted Traffic Load')
    plt.title('Actual vs. Predicted Traffic Load')
    plt.show()

In [ ]:
%%q
.pykx.get[`model] tw

## What Next? (out of scope)
* What if performance problems arise in the Python libraries?
* Exploit KX ecosystem (ml library, dashboards, etc.)
* Implement your own functionality & contribute!